In [128]:
import networkx as nx

from optimal_splitting.src.generate_AS_network import generate_directed_AS_graph, graph_pruning_via_BFS
from optimal_splitting.src.auxiliary_functions import gen_pyvis_network

In [135]:
# global params
nr_ASes = 200
nr_allies = 2

In [136]:
# graph generation

G_init, victim, adversary, allies = generate_directed_AS_graph(nr_ASes, nr_allies)

# prune some edges of it
G_pruned = graph_pruning_via_BFS(G_init, victim)

In [137]:
net = gen_pyvis_network(G_pruned)
net.show("test.html")

In [142]:
def identify_attack_flows(
    initial_Graph:nx.classes.graph.Graph,
    victim:int,
    adversary:int
    ):
    """
    TODO.
    
    
    :param initial_Graph: graph
    :param victim: the victim node
    :param adversary: the adversary node
    
    :type initial_Graph: nx.classes.graph.Graph
    :type victim: int
    :type adversary: int
    
    :return: a tuple containing (in order):
        * the graph with the added flow information on nodes and edges
        * a list of all nodes in the attack path, ordered from closes to adversary to farthest
    :rtype: tuple
    """

    graph = initial_Graph.copy()
    
    attack_flow_nodes = []
    
    # set a attribute denoting the path taken
    for node_indx in graph.nodes:
        graph.nodes[node_indx]["AS_paths"] = []
    for u, v in graph.edges:
        graph[u][v]["AS_paths"] = []

    # start a queue
    Q = [adversary]
    graph.nodes[adversary]["AS_paths"] = [[adversary]]

    while Q:
        # get the next node
        current_node = Q.pop(0)
        attack_flow_nodes.append(current_node)
        
        # consider all its outward pointing edges
        for u, v in graph.out_edges(current_node):
            # note the flows that path through this note
            graph[u][v]["AS_paths"] = graph.nodes[current_node]["AS_paths"]

            # if the next node is not the victim node, then add it
            graph.nodes[v]["AS_paths"].extend([path + [v] for path in graph[u][v]["AS_paths"]])
            # make it unique
            graph.nodes[v]["AS_paths"] = [list(x) for x in set(tuple(x) for x in graph.nodes[v]["AS_paths"])]

            # add the node to the queue, if it is node the victim node
            if v != victim:
                Q.append(v)

    return graph, list(set(attack_flow_nodes))


In [150]:
G_identified, attack_flow_nodes = identify_attack_flows(G_pruned, victim, adversary)

In [151]:
attack_flow_nodes

[2, 5, 6, 8, 9, 12, 13, 110, 19, 23]

In [159]:

def add_shortest_distances(
    initial_Graph:nx.classes.graph.Graph,
    attack_flow_nodes:list,
    allies:list
    ):
    """
    This function will calculate the shortest distance to all ally nodes from all
    nodes that are on the attack flow and add this information as a node attribute
    to them. Note, that this information is available to all of these nodes in the
    AS network through the length of the AS_PATH path attribute. This information
    is a simulation of this information.

    :param initial_Graph: graph
    :param attack_flow_nodes: a list of all nodes that are on the attack path
    :param allies: list of all allies

    :type initial_Graph: nx.classes.graph.Graph
    :type attack_flow_nodes: list
    :type allies: list

    :return: the graph with added shortest distance information on attack path nodes
    :rtype: nx.classes.graph.Graph
    """

    graph = initial_Graph.copy()
    undirected_graph = initial_Graph.to_undirected()

    for attack_flow_node in attack_flow_nodes:
        # create a dictionary for saving the shortest distances to all allies
        distances = {}

        for ally in allies:
            # calculate the shorteset distance and note it down
            distances[ally] = len(list(nx.all_shortest_paths(undirected_graph.to_undirected(), attack_flow_node, ally))[0])

        # save this ditionary as a node attribute
        graph.nodes[attack_flow_node]["ally_distances"] = distances

    return graph

In [161]:
G_shortest = add_shortest_distances(G_identified, attack_flow_nodes, allies)
allies

[174, 37]

In [163]:
G_shortest.nodes[2]

{'type': 'T',
 'color': 'darkgrey',
 'value': 30,
 'size': 10,
 'AS_paths': [[110, 12, 2]],
 'ally_distances': {174: 3, 37: 5}}

In [164]:
list(nx.all_shortest_paths(G_identified.to_undirected(), 2, 174)) 

[[2, 12, 174]]